# APIM을 이용한 로드 밸런싱

Azure의 PAAS형 리소스를 배포하면 해당 리소스를 호출해 사용할 수 있는 엔드포인트가 생성됩니다. 이때 각 리소스의 사용 쿼타(Quota)는 한정돼있으므로, 엔드포인트를 그 이상으로 호출한다면 Error Code 429가 리턴될 것입니다.

예를 들어 Azure OpenAI는 Region별로 TPM(Tokens Per Minute)이 정해져있기 때문에, Azure OpenAI 엔드포인트를 짧은 시간 내 반복적으로 호출한다면 429가 리턴되며 원하는 답변을 얻지 못할 가능성이 존재합니다.

APIM은 이러한 상황에 대한 해결책이 될 수 있습니다. 미리 여러 Region에 동일한 deployment name과 model type의 Azure OpenAI 리소스를 배포해두고, APIM에 해당 백엔드 엔드포인트를 등록해 Inbound Policy로 Failover / Load Balancing을 수행할 수 있습니다.

본 노트북으로 APIM을 테스트할 수 있습니다.

Azure OpenAI, APIM 배포에 대한 가이드는 [Microsoft Korea 테크블로그 (TBU)](link_TBU)에서 확인할 수 있습니다.

### Prerequisite

**Ensure you have an .env file containing:**

APIM_URL = ""

APIM_SUBSCRIPTION_KEY = ""

APIM_CHAT_DEPLOYMENT_NAME = ""

APIM_EMBEDDING_DEPLOYMENT_NAME = ""

APIM_API_VERSION = ""

*

아래의 셀을 실행해 .env 파일을 생성할 수 있습니다.

In [ ]:
%%writefile .env

APIM_URL = ""
APIM_SUBSCRIPTION_KEY = ""
APIM_CHAT_DEPLOYMENT_NAME = ""
APIM_EMBEDDING_DEPLOYMENT_NAME = ""
APIM_API_VERSION = ""

# Run APIM Tests

In [1]:
import sys
import os
import requests

#Load the env details
from dotenv import load_dotenv
load_dotenv()

True

### Single Call (Test Connection)

In [2]:
# Get environment variables
apim_url = os.getenv("APIM_URL")
api_version = os.getenv("APIM_API_VERSION")
subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY")

embedding_deployment_name = os.getenv("APIM_EMBEDDING_DEPLOYMENT_NAME")
chat_deployment_name = os.getenv("APIM_CHAT_DEPLOYMENT_NAME")

# Construct the URL and headers
embedding_url = f'{apim_url}/deployments/{embedding_deployment_name}/embeddings?api-version={api_version}'
chat_url = f'{apim_url}/deployments/{chat_deployment_name}/chat/completions?api-version={api_version}'

headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

In [5]:
# Test Connection: EMBEDDING

# Sample input text
input_text = "Hello, World"

# Make the request
response = requests.post(embedding_url, headers=headers, json={"input": input_text})

# Print the response
print(f"Response status code: {response.status_code}")

# Check if the request was successful
if response.status_code == 200:
    embedding = response.json().get("data", [])[0].get("embedding", [])
    print(f"Embedding: {embedding}")
else:
    print("Failed to get embedding")

Response status code: 200
Embedding: [-0.013374719, -0.002001362, -0.012883666, -0.038948216, -0.005511418, 0.011455737, -0.018517848, -0.011016374, -0.0066873594, -0.027912457, 0.030987997, 0.012696291, -0.016760396, -0.009517373, 0.017962182, 0.012256929, 0.023828968, -0.007507934, 0.008561112, 0.012185855, -0.0067196656, 0.0005613182, 0.0126381405, -0.0011008298, -0.010589934, -0.014731574, 0.00032427232, -0.028506888, 0.034528743, -0.038586386, 0.0045680804, -0.009258924, -0.0058280174, -0.026090393, 0.009530295, -0.019138124, -0.003547208, -0.017186837, 0.02368682, -0.016812086, 0.0049945205, 0.016437335, 0.0019900547, -0.022006905, -0.032616224, 0.013684858, 0.012928895, -0.00929123, -0.0011985557, 0.021244481, 0.02362221, -0.0040156464, -0.018453235, -0.0048362208, -0.007960219, 0.0013100117, -0.02910132, 0.024966141, 0.025534729, -0.022588413, 0.01550692, 0.007023343, -0.016282266, 0.007126722, -0.014796186, 0.010518861, 0.013788236, 0.013568555, -0.017755425, 0.018789219, 0.02

In [7]:
# Test Connection: CHAT

# Sample input text
input_text = {
        "messages": [
            {
                "role": "user",
                "content": "Hey. What's up?"
            }
        ],
    }

# Make the request
response = requests.post(chat_url, headers=headers, json=input_text)

# Print the response
print(f"Response status code: {response.status_code}")

# Check if the request was successful
if response.status_code == 200:
    chat = response.json()['choices'][0]['message']['content']
    print(f"Chat: {chat}")
else:
    print("Failed to get chat")

Response status code: 200
Chat: Hey there! Not much, just here and ready to chat or help with whatever you need. What's up with you? 😊


### Multiple Calls (Test Failover)

In [6]:
# Test Failover: EMBEDDING

input_text = "Hello, World!"

total_calls = 30
failure = 0
failure_reasons = {}

for i in range(total_calls):
    response = requests.post(embedding_url, headers=headers, json={"input": input_text})
    if response.status_code == 200:
        embedding = response.json().get("data", [])[0].get("embedding", [])[0]
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: {embedding} ...")
    else:
        failure += 1
        failure_reasons.update({response.status_code: i})   # Keep track of different failure http code returned
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: Failed to get embedding")
        

print(f"\n{(total_calls-failure)} succeeded out of {total_calls} calls")
if failure > 0:
    print(f"\n{failure} failure with response {', '.join(map(str, failure_reasons.keys()))}")

[1/30] Status 200: -0.0026247576 ...
[2/30] Status 200: -0.0026247576 ...
[3/30] Status 200: -0.0026247576 ...
[4/30] Status 200: -0.0026247576 ...
[5/30] Status 200: -0.0026247576 ...
[6/30] Status 200: -0.0026247576 ...
[7/30] Status 200: -0.0026431093 ...
[8/30] Status 200: -0.0026247576 ...
[9/30] Status 200: -0.0026431093 ...
[10/30] Status 200: -0.0026247576 ...
[11/30] Status 200: -0.0026431093 ...
[12/30] Status 200: -0.0026431093 ...
[13/30] Status 200: -0.0026247576 ...
[14/30] Status 200: -0.0026247576 ...
[15/30] Status 200: -0.0026431093 ...
[16/30] Status 200: -0.0026247576 ...
[17/30] Status 200: -0.0026247576 ...
[18/30] Status 200: -0.0026247576 ...
[19/30] Status 200: -0.0026431093 ...
[20/30] Status 200: -0.0026247576 ...
[21/30] Status 200: -0.0026431093 ...
[22/30] Status 200: -0.0026247576 ...
[23/30] Status 200: -0.0026247576 ...
[24/30] Status 200: -0.0026431093 ...
[25/30] Status 200: -0.0026247576 ...
[26/30] Status 200: -0.0026247576 ...
[27/30] Status 200: -

In [8]:
# Test Failover: CHAT

input_text = {
        "messages": [
            {
                "role": "user",
                "content": "Hey. What's up?"
            }
        ],
    }

total_calls = 30
failure = 0
failure_reasons = {}

for i in range(total_calls):
    response = requests.post(chat_url, headers=headers, json=input_text)
    if response.status_code == 200:
        chat = response.json()['choices'][0]['message']['content'][:6]
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: {chat} ...")
    else:
        failure += 1
        failure_reasons.update({response.status_code: i})   # Keep track of different failure http code returned
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: Failed to get chat")

print(f"\n{(total_calls-failure)} succeeded out of {total_calls} calls")
if failure > 0:
    print(f"\n{failure} failure with response {', '.join(map(str, failure_reasons.keys()))}")

[1/30] Status 200: Hello! ...
[2/30] Status 200: Hey! N ...
[3/30] Status 200: Hey th ...
[4/30] Status 200: Hey th ...
[5/30] Status 200: Hey th ...
[6/30] Status 200: Hey! N ...
[7/30] Status 200: Hey th ...
[8/30] Status 200: Hey th ...
[9/30] Status 200: Hey th ...
[10/30] Status 200: Hello! ...
[11/30] Status 200: Hello! ...
[12/30] Status 200: Hey th ...
[13/30] Status 200: Hey th ...
[14/30] Status 200: Hey th ...
[15/30] Status 200: Hey th ...
[16/30] Status 200: Hello! ...
[17/30] Status 200: Hey th ...
[18/30] Status 200: Hello! ...
[19/30] Status 200: Hey th ...
[20/30] Status 200: Hello! ...
[21/30] Status 200: Hey th ...
[22/30] Status 200: Hey th ...
[23/30] Status 200: Hey th ...
[24/30] Status 200: Hey th ...
[25/30] Status 200: Hey th ...
[26/30] Status 200: Hey th ...
[27/30] Status 200: Hello! ...
[28/30] Status 200: Hey th ...
[29/30] Status 200: Hey th ...
[30/30] Status 200: Hello! ...

30 succeeded out of 30 calls


### Check Log

APIM 리소스의 Logs 항목에서 구체적인 로그를 확인할 수 있습니다.

다음 KQL 쿼리를 실행해 BackendUrl의 내용을 확인해보면, 실제로 엔드포인트가 바뀌는 것을 확인할 수 있습니다.

```
ApiManagementGatewayLogs
| project TimeGenerated, BackendResponseCode, BackendUrl, BackendTime
| order by TimeGenerated desc
| top 60 by TimeGenerated
```

백엔드 1에서 429(Too Many Requests)가 발생한다면, APIM이 백엔드 2로 retry를 해 200(Success)을 리턴받게 되는 것입니다.

*apim_log.png 참고*